In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

sns.set()
warnings.simplefilter('ignore')

In [ ]:
data_orig = pd.read_csv('/kaggle/input/water-potability/water_potability.csv')
print(data_orig.shape)
data_orig.head()

In [ ]:

#  1) pH value: PH is an important parameter in evaluating the acid–base balance of water. It is also the indicator of acidic or alkaline condition of water status. WHO has recommended maximum permissible limit of pH from 6.5 to 8.5. The current investigation ranges were 6.52–6.83 which are in the range of WHO standards.
#  2) Hardness: Hardness is mainly caused by calcium and magnesium salts. These salts are dissolved from geologic deposits through which water travels. The length of time water is in contact with hardness producing material helps determine how much hardness there is in raw water. Hardness was originally defined as the capacity of water to precipitate soap caused by Calcium and Magnesium.
#  3) Solids (Total dissolved solids - TDS): Water has the ability to dissolve a wide range of inorganic and some organic minerals or salts such as potassium, calcium, sodium, bicarbonates, chlorides, magnesium, sulfates etc. These minerals produced un-wanted taste and diluted color in appearance of water. This is the important parameter for the use of water. The water with high TDS value indicates that water is highly mineralized. Desirable limit for TDS is 500 mg/l and maximum limit is 1000 mg/l which prescribed for drinking purpose.
#  4) Chloramines: Chlorine and chloramine are the major disinfectants used in public water systems. Chloramines are most commonly formed when ammonia is added to chlorine to treat drinking water. Chlorine levels up to 4 milligrams per liter (mg/L or 4 parts per million (ppm)) are considered safe in drinking water.
#  5) Sulfate: Sulfates are naturally occurring substances that are found in minerals, soil, and rocks. They are present in ambient air, groundwater, plants, and food. The principal commercial use of sulfate is in the chemical industry. Sulfate concentration in seawater is about 2,700 milligrams per liter (mg/L). It ranges from 3 to 30 mg/L in most freshwater supplies, although much higher concentrations (1000 mg/L) are found in some geographic locations.
#  6) Conductivity: Pure water is not a good conductor of electric current rather’s a good insulator. Increase in ions concentration enhances the electrical conductivity of water. Generally, the amount of dissolved solids in water determines the electrical conductivity. Electrical conductivity (EC) actually measures the ionic process of a solution that enables it to transmit current. According to WHO standards, EC value should not exceeded 400 μS/cm.
#  7) Organic_carbon: Total Organic Carbon (TOC) in source waters comes from decaying natural organic matter (NOM) as well as synthetic sources. TOC is a measure of the total amount of carbon in organic compounds in pure water. According to US EPA < 2 mg/L as TOC in treated / drinking water, and < 4 mg/Lit in source water which is use for treatment.
#  8) Trihalomethanes: THMs are chemicals which may be found in water treated with chlorine. The concentration of THMs in drinking water varies according to the level of organic material in the water, the amount of chlorine required to treat the water, and the temperature of the water that is being treated. THM levels up to 80 ppm is considered safe in drinking water.
#  9) Turbidity: The turbidity of water depends on the quantity of solid matter present in the suspended state. It is a measure of light emitting properties of water and the test is used to indicate the quality of waste discharge with respect to colloidal matter. The mean turbidity value obtained for Wondo Genet Campus (0.98 NTU) is lower than the WHO recommended value of 5.00 NTU.
#  10) Potability: Indicates if water is safe for human consumption where 1 means Potable and 0 means Not potable.


In [ ]:
data = data_orig.copy()
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
feat_cols = ['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity','Organic_carbon', 'Trihalomethanes', 'Turbidity']
output_col = 'Potability'

In [ ]:
fig,ax1 = plt.subplots(5,2,figsize = (20,20))
k = 0
skew_vals = data[feat_cols].skew()
for i in range(5):
    if i != 4 :
        for j in range(2):
            sns.histplot(data=data,x = feat_cols[k],ax = ax1[i][j],label = '{:.4}'.format(skew_vals[k]))
            ax1[i][j].legend()
            k += 1
    else:
        sns.histplot(data=data,x = feat_cols[k],ax = ax1[i][0],label = '{:.4}'.format(skew_vals[k]))
plt.show()

In [ ]:
%%capture
'''
All columns are almost normally distributed
Solids has the highest skewness value
'''

In [ ]:
fig = plt.figure(figsize = (10,7))
plt.pie(data[output_col].value_counts(),labels = ['Not Potable','Potable'],autopct='%1.1f%%');

In [ ]:
## checking for outliers
fig,ax2 = plt.subplots(5,2,figsize = (30,30))
k = 0
q1 = data[feat_cols].quantile(0.25)
q3 = data[feat_cols].quantile(0.75)
iqr = q3 - q1
upper_lim = dict(q3 + 1.5*iqr)
lower_lim = dict(q1 - 1.5*iqr)

for i in range(5):
    if i != 4:
        for j in range(2):
            col = feat_cols[k]
            out_count = sum(data[col] > upper_lim[col]) + sum(data[col] < lower_lim[col])
            out_prop = out_count / len(data)
            sns.boxplot(data=data,x = feat_cols[k],ax = ax2[i][j])
            ax2[i][j].set_title('Outliers Proporation : {:.2f}%'.format(out_prop*100))
            k += 1
    else:
        col = feat_cols[k]
        out_count = sum(data[col] > upper_lim[col]) + sum(data[col] < lower_lim[col])
        out_prop = out_count / len(data)
        sns.boxplot(data=data,x = feat_cols[k],ax = ax2[i][0])
        ax2[i][0].set_title('Outliers Proporation : {:.2f}%'.format(out_prop*100))
plt.show()

In [ ]:
%%capture
'''
Most columns contain outliers that are normally distributed except Solids column
Will outliers effect model performance? ,(can they be safely replaced?)

Since columns ['ph','sulfates','trihalomethanes'] have outliers and are normally distributed
will replace missing values with median/mode.
'''

In [ ]:
sns.pairplot(data = data,hue = output_col)

In [ ]:
corr = data.corr()
sns.clustermap(corr,annot = True,cmap = 'Greens')
plt.show()

In [ ]:
### Data Preprocessing ###

## Fill missing values
miss_data_cols = ['ph','Sulfate','Trihalomethanes']
for col in miss_data_cols:
    data[col] = data[col].fillna(data[col].median())
    
data.isnull().sum()

In [ ]:
## remove outliers 

# create new copy for data with replaced outliers
data1 = data.copy()
for col in feat_cols:
    data1[col][data1[col] > upper_lim[col]] = upper_lim[col]
    data1[col][data1[col] < lower_lim[col]] = lower_lim[col]

In [ ]:
# preprocessing
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import MinMaxScaler,StandardScaler

# models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# metrics
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [ ]:
def preprocess_data(data,feat_cols,output_col,scaler,scale_op = 'fit',train_size = 0.7):
    X = data[feat_cols].values
    Y = data[output_col].values
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = (1 - train_size))
    if scale_op == 'fit':
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
    elif scale_op == 'transform':
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
    else:
        print('Not scaling')
    return {'X_train' : X_train,'X_test' : X_test,'Y_train': Y_train,'Y_test':Y_test}

def train_model(model_name,model,data_dict):
    model.fit(data_dict['X_train'],data_dict['Y_train'])
    preds = model.predict(data_dict['X_test'])
    print('---------------------- Report ------------------------')
    print('Model : {}'.format(model_name))
    print(classification_report(data_dict['Y_test'],preds))
    return model

In [ ]:
models_dict = {
    'Logistic Regression' : LogisticRegression(random_state=42),
    'Decision Tree' : DecisionTreeClassifier(),
    'Linear SVM' : SVC(kernel = 'linear',random_state = 42),
    'RBF SVM' : SVC(kernel = 'rbf',random_state=42),
    'Random Forest' : RandomForestClassifier(n_estimators=100,random_state=42),
}

In [ ]:
stScaler = StandardScaler()
data_dict = preprocess_data(data1,feat_cols,output_col,stScaler,scale_op = 'fit')
print(data_dict['X_train'].shape,data_dict['Y_train'].shape)
print(data_dict['X_test'].shape,data_dict['Y_test'].shape)

In [ ]:
trained_models_dict = {}
for model_name,model in models_dict.items():
    trained_models_dict[model_name] = train_model(model_name,model,data_dict)

In [ ]:
%%capture
'''
Most models have a comparitively low f1 score for class 1 (Potable) compared to class 0 (Non potable)

Random Forest Classifier performs best from all the models
'''